In [ ]:
# ==========================================
# 🧠 CAMPUS CONNECT: SECURE CLOUD AI SERVER
# ==========================================

# 1. Install Dependencies
!pip install fastapi uvicorn pyngrok python-multipart transformers torch pillow nest_asyncio

import uvicorn
import getpass
import os
from fastapi import FastAPI, UploadFile, File
from pyngrok import ngrok
from transformers import pipeline
from PIL import Image
import io
import nest_asyncio

# 2. Load the BLIP Model (Downloads to Colab GPU)
print("⏳ Loading BLIP Model... (This takes ~30s)")
captioner = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base", device=0)
print("✅ BLIP Model Loaded & Ready!")

# 3. Define the API
app = FastAPI()

@app.post("/analyze")
async def analyze(file: UploadFile = File(...)):
    try:
        # Read the uploaded image
        image_data = await file.read()
        image = Image.open(io.BytesIO(image_data))

        # Run Inference
        result = captioner(image, max_new_tokens=20)
        text = result[0]['generated_text']

        return {"description": text}
    except Exception as e:
        return {"error": str(e)}

# 4. SECURE TOKEN INPUT
print("👇 Paste your Ngrok Authtoken in the box below and hit Enter:")
NGROK_AUTH_TOKEN = getpass.getpass()

ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# 5. Open the Tunnel
ngrok.kill()
public_url = ngrok.connect(8000).public_url

print("="*60)
# This is the URL you need to copy to your laptop
print(f"🚀 YOUR PUBLIC URL IS:  {public_url}/analyze")
print("="*60)

# 6. Start the Server (FIXED FOR COLAB)
# We use 'await' here to respect the notebook's environment
config = uvicorn.Config(app, port=8000)
server = uvicorn.Server(config)
await server.serve()